# U-Net Model for RV Segmentation

## Training U-Net CNN

In [1]:
import sys
sys.path.append("..")
import os, re, glob
import numpy as np
import matplotlib as mplt
%matplotlib inline

from src import data,unet

Using TensorFlow backend.


In [2]:
train_dir = "/Users/aksharkumar/Documents/mlndCapstone/trainingdata/TrainingSet"


train_images=[]
train_inner_masks=[]
train_outer_masks = []

patient_directories = sorted(glob.glob(os.path.join(train_dir, "patient*")))

for patient_dir in patient_directories:
    imgdata = data.ImageData(patient_dir)
    train_images += imgdata.labeled_images
    train_inner_masks += imgdata.endo_masks.values()
    train_outer_masks += imgdata.epi_masks.values()

print("There are %d total training images." % len(train_images))
print("There are %d total inner masks." % len(train_inner_masks))
print("There are %d total outer masks." % len(train_outer_masks))


There are 243 total training images.
There are 243 total inner masks.
There are 243 total outer masks.


In [3]:
model = unet.UNet().get_unet(height=216,width=256,channels=1,features=64,steps=4)



ValueError: Negative dimension size caused by subtracting 3 from 1 for 'conv2d_11/convolution' (op: 'Conv2D') with input shapes: [?,1,0,1024], [3,3,1024,512].